In [3]:
%pip install asyncpg
%pip install python-dotenv
%pip install pandas

  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/568.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/568.0 kB ? eta -:--:--
    --------------------------------------- 10.2/568.0 kB ? eta -:--:--
    --------------------------------------- 10.2/568.0 kB ? eta -:--:--
    --------------------------------------- 10.2/568.0 kB ? eta -:--:--
   -- ------------------------------------ 30.7/568.0 kB 145.2 kB/s eta 0:00:04
   -- ------------------------------------ 30.7/568.0 kB 145.2 kB/s eta 0:00:04
   -- ------------------------------------ 41.0/568.0 kB 140.3 kB/s eta 0:00:04
   -- ------------------------------------ 41.0/568.0 kB 140.3 kB/s eta 0:00:04
   ---- ---------------------------------- 61.4/568.0 kB 163.6 kB/s eta 0:00:04
   ---- ---------------------------------- 71.7/568.0 kB 157.1 kB/s eta 0:00:04
   ------ -------------------------------- 92.2/568.0 kB 187.0 kB/s eta 0:00:03
   ------ --

In [4]:
# Import the necessary libraries
import pandas as pd
import asyncpg
import os
from dotenv import load_dotenv

In [ ]:
# Load variables from .env file
load_dotenv()

# Retrieve the environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

In [ ]:
async def fetch_intelligence_data_model(table_name='intelligence_data_model'):
    try:
        # Create a connection pool
        pool = await asyncpg.create_pool(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
            min_size=1,
            max_size=10
        )

        # Use the pool for database operations
        async with pool.acquire() as connection:
            # Fetch column names from the table
            columns_query = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';"
            columns = await connection.fetch(columns_query)
            column_names = [col[0] for col in columns if col[0]]

            # Fetch data from the table
            data_query = f"""SELECT * FROM {table_name};"""
            result = await connection.fetch(data_query)

            # Create a DataFrame with fetched data and column names
            intelligence_data_model_df = pd.DataFrame(result, columns=column_names)

            return intelligence_data_model_df

    except Exception as e:
        print("Error:", e)

In [ ]:
intelligence_data_model_df = await fetch_intelligence_data_model()

In [ ]:
intelligence_data_model_df

,dutch_names,attribute,current_category,vioscore,dimension,table_name
0,Opleidingsniveau laag,education_level_low,None,NonVioScore,Credit,all_gemeente_data_view
1,Diefstal,theft,None,NonVioScore,Environment,all_gemeente_data_view
2,Hotel afstand,hotel_distance,None,NonVioScore,Credit,all_gemeente_data_view
3,Voldoet aan alcoholrichtlijn,meets_alcohol_guideline,drinking,VioScore,Health,all_health_data_view
4,Drinker,drinker,drinking,VioScore,Health,all_health_data_view
...,...,...,...,...,...,...
384,None,RBI,interest_rate,VioScore,World,world_data
385,None,CBR,interest_rate,VioScore,World,world_data
386,None,AirQuality,weather,VioScore,Environment,weather_data
387,None,UVIndex,weather,VioScore,Environment,weather_data


In [ ]:
async def fetch_data_async(table_name: str,
                           attribute_name: str,
                           region_code: str = 'NL00'
                           ):
    try:
        # Create a connection pool
        pool = await asyncpg.create_pool(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
            min_size=1,
            max_size=10
        )

        # Use the pool for database operations
        async with pool.acquire() as connection:
            # Fetch column names from the table
            columns_query = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';"
            columns = await connection.fetch(columns_query)
            column_names = [col[0] for col in columns if col[0]==attribute_name]

            # Fetch data from the table
            data_query = f"""SELECT "{attribute_name}" FROM {table_name} WHERE "Regiocode" = {"'{}'".format(region_code)};"""
            result = await connection.fetch(data_query)

            # Create a DataFrame with fetched data and column names
            intelligence_data_model_df = pd.DataFrame(result, columns=column_names)

            return intelligence_data_model_df

    except Exception as e:
        print("Error:", e)

In [ ]:
import pandas as pd

# Import the CSV file
# data = pd.read_csv('data/Data_2020_65_jaar_of_ouder.csv')

# Import health data from database
health_data = await fetch_data_async('all_health_data_view',
                                     'weekly_athletes'.replace("_", " ").title(),
                                     region_code='PV21')

health_data

Error: column "Weekly Athletes" does not exist


In [ ]:
import numpy as np

In [ ]:
async def generate_intelligence_view(region_code: str='NL00',
                                     region_name: str='Nederland',
                                     region_type: str='Country'):
  i = 1
  intelligence_dictionary = {
      "labels": [
          region_type,
          "Region"
      ],
      "index": f"{i}",
      "code": region_code,
      "name": region_name,
      "children": [],
      "vioscore": '@VioScore Placeholder'
      }


  j = 1
  for vioscore_type in intelligence_data_model_df["vioscore"].unique():
    vioscore_dict = {
        'labels': [
            vioscore_type,
            "VioScoreTotal"
            ],
        'index': f'{i}.{j}',
        'code': region_code,
        'vioscore': '@VioScore Placeholder',
        'children': [

        ]
    }
    intelligence_dictionary['children'].append(vioscore_dict)
    vioscore_intelligence_data_model_df = intelligence_data_model_df[intelligence_data_model_df['vioscore'] == vioscore_type]

    k = 1
    for dimension in vioscore_intelligence_data_model_df['dimension'].unique():
        dimension_dict = {
            'labels': [
                dimension,
                'Dimension'
            ],
            'index': f'{i}.{j}.{k}',
            'code': region_code,
            'vioscore': '@VioScore Placeholder',
            'children': [

            ]
        }
        vioscore_dict['children'].append(dimension_dict)
        dimension_intelligence_data_model_df = vioscore_intelligence_data_model_df[vioscore_intelligence_data_model_df['dimension'] == dimension]

        l = 1
        for category in dimension_intelligence_data_model_df['current_category'].unique():
          if category != None:
            category_dict = {
              'labels': [
                  category.replace("_", " ").title().replace(" ", ""),
                  'Category'
              ],
              'code': region_code,
              'vioscore': '@VioScore Placeholder',
              'index': f'{i}.{j}.{k}.{l}',
              'children': []
            }
            dimension_dict['children'].append(category_dict)

          category_intelligence_data_model_df = dimension_intelligence_data_model_df[dimension_intelligence_data_model_df['current_category'] == category]

          m = 1
          for attribute in category_intelligence_data_model_df['attribute'].unique():
            dutch_column_name = category_intelligence_data_model_df[category_intelligence_data_model_df['attribute'] == attribute]['dutch_names'].values[0]
            table_name = category_intelligence_data_model_df[category_intelligence_data_model_df['attribute'] == attribute]['table_name'].values[0]
            try:
              coroutine = await fetch_data_async(table_name, # table_name placeholder
                                                  dutch_column_name,
                                                  region_code='PV21')
              attribute_score = coroutine.values[0]
            except:
              attribute_score = 0
            if attribute != None:
              attribute_dict = {
                  'labels': [
                      attribute.replace("_", " ").title().replace(" ", ""),
                      'Attribute'
                  ],
                  'code': region_code,
                  'index': f"{i}.{j}.{k}.{l}.{m}",
                  'vioscore': attribute_score[0] if type(attribute_score)==np.ndarray else attribute_score,
                  'children': []
              }
              category_dict['children'].append(attribute_dict)
            m += 1
          l += 1
        k += 1
    j += 1
  i += 1
  return intelligence_dictionary

In [ ]:
dictionary = await generate_intelligence_view()

Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many clients already
Error: sorry, too many cl

In [ ]:
dictionary

{'labels': ['Country', 'Region'],
 'index': '1',
 'code': 'NL00',
 'name': 'Nederland',
 'children': [{'labels': ['NonVioScore', 'VioScoreTotal'],
   'index': '1.1',
   'code': 'NL00',
   'vioscore': '@VioScore Placeholder',
   'children': [{'labels': ['Credit', 'Dimension'],
     'index': '1.1.1',
     'code': 'NL00',
     'vioscore': '@VioScore Placeholder',
     'children': []},
    {'labels': ['Environment', 'Dimension'],
     'index': '1.1.2',
     'code': 'NL00',
     'vioscore': '@VioScore Placeholder',
     'children': []},
    {'labels': ['Health', 'Dimension'],
     'index': '1.1.3',
     'code': 'NL00',
     'vioscore': '@VioScore Placeholder',
     'children': [{'labels': ['PhysicalActivity', 'Category'],
       'code': 'NL00',
       'vioscore': '@VioScore Placeholder',
       'index': '1.1.3.1',
       'children': [{'labels': ['WalkAndOrCycleToSchoolOrWork', 'Attribute'],
         'code': 'NL00',
         'index': '1.1.3.1.1',
         'vioscore': None,
         'children

In [ ]:
# Vioscore Per Category
import asyncpg
import pandas as pd
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

# Retrieve the environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

async def fetch_data_async(table_name):
    try:
        # Create a connection pool
        pool = await asyncpg.create_pool(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
            min_size=1,
            max_size=5
        )

        # Use the pool for database operations
        async with pool.acquire() as connection:
            # Fetch data from the 'world_data' table
            data_query = f"SELECT * FROM {table_name};"
            result = await connection.fetch(data_query)

            # Get the column names
            column_names = result[0].keys()

            # Create a DataFrame with fetched data and column names
            df = pd.DataFrame(result, columns=column_names)

            return df

    except Exception as e:
        print("Error:", e)


In [ ]:
import pandas as pd
import numpy as np

intelligence_data_model = await fetch_data_async('intelligence_data_model')

categories = intelligence_data_model[(intelligence_data_model['dimension'] == 'Health')]['current_category'].unique().tolist()

while None in categories:
    categories.remove(None)


Error: password authentication failed for user "superuser"


In [ ]:
health_vioscore_table = await fetch_data_async('health_vioscore_table')

In [ ]:
def get_values_for_region(region_code, categories):

    # Filter the DataFrame based on the region code
    filtered_df = health_vioscore_table[health_vioscore_table['region_code'] == region_code]
    vioscore_by_category = {}
    for category in categories:
        attributes_to_select = intelligence_data_model[(intelligence_data_model['current_category'] == category)]['attribute'].tolist()

        if category == 'drinking':
            values_for_region = filtered_df[attributes_to_select]
            meets_alcohol_guideline = values_for_region['meets_alcohol_guideline']
            drinker = values_for_region['drinker']
            heavy_drinker = values_for_region['heavy_drinker']
            excessive_drinker = values_for_region['excessive_drinker']

            alcohol_avg = (((1 - drinker) + (1 - heavy_drinker) + (1 - excessive_drinker) + meets_alcohol_guideline) / 4) * 1000
            vioscore_by_category[category] = float(alcohol_avg.iloc[0])

        elif category == 'weight':
            values_for_region = filtered_df[attributes_to_select]
            underweight = values_for_region['underweight']
            normal_weight = values_for_region['normal_weight']
            overweight = values_for_region['overweight']
            severe_obesity = values_for_region['severe_obesity']

            weight_avg = (((1 - underweight) + (1 - overweight) + (1 - severe_obesity) + normal_weight) / 4) * 1000
            vioscore_by_category[category] = float(weight_avg.iloc[0])

        elif category == 'smoker':
            values_for_region = filtered_df[attributes_to_select]
            smoker = values_for_region['smoker']

            smoker_avg = (1 - smoker) * 1000
            vioscore_by_category[category] = float(smoker_avg.iloc[0])

        elif category == 'physical_activity':
            # These are manually defined because the intelligence data model has a few more added attributes
            attributes_to_select = ['meets_exercise_guideline', 'weekly_athletes']
            values_for_region = filtered_df[attributes_to_select]
            meets_exercise_guideline = values_for_region['meets_exercise_guideline']
            weekly_athletes = values_for_region['weekly_athletes']

            physical_activity_avg = ((meets_exercise_guideline + weekly_athletes) / 2) * 1000
            vioscore_by_category['physical_activity_avg'] = float(physical_activity_avg.iloc[0])

        elif category == 'physical_health':
            values_for_region = filtered_df[attributes_to_select]
            good_perceived_health = values_for_region['good_perceived_health']
            prolonged_illness_and_limited = values_for_region['prolonged_illness_and_limited']

            physical_health_avg = (((1 - prolonged_illness_and_limited) + good_perceived_health) / 2) * 1000
            vioscore_by_category[category] = float(physical_health_avg.iloc[0])

        elif category == 'impairment':
            values_for_region = filtered_df[attributes_to_select]
            one_or_more_long_term_conditions = values_for_region['one_or_more_long_term_conditions']
            restricted_due_to_health = values_for_region['restricted_due_to_health']
            severely_restricted_due_to_health = values_for_region['severely_restricted_due_to_health']
            hearing_impairment = values_for_region['hearing_impairment']
            face_restriction = values_for_region['face_restriction']
            mobility_restriction = values_for_region['mobility_restriction']
            one_or_more_physical_limitations = values_for_region['one_or_more_physical_limitations']

            impairment_avg = (((1 - one_or_more_long_term_conditions) + (1 - restricted_due_to_health) + (1 - severely_restricted_due_to_health) + (1 - hearing_impairment) + (1 - face_restriction) + (1 - mobility_restriction) + (1 - one_or_more_physical_limitations)) / 7) * 1000
            vioscore_by_category[category] = float(impairment_avg.iloc[0])

        elif category == 'loneliness':
            attributes_to_select = ['lonely', 'severely_or_very_seriously_lonely']
            values_for_region = filtered_df[attributes_to_select]
            lonely = values_for_region['lonely']
            severely_very_seriously_lonely = values_for_region['severely_or_very_seriously_lonely']

            loneliness_avg = (((1 - lonely) + (1 - severely_very_seriously_lonely)) / 2) * 1000
            vioscore_by_category[category] = float(loneliness_avg.iloc[0])

        elif category == 'caregiving':
            values_for_region = filtered_df[attributes_to_select]
            volunteer_work = values_for_region['volunteer_work']
            caregiver = values_for_region['caregiver']

            caregiving_avg = ((volunteer_work + caregiver) / 2) * 1000
            vioscore_by_category[category] = float(caregiving_avg.iloc[0])

        elif category == 'stress':
            values_for_region = filtered_df[attributes_to_select]
            moderate_or_much_control_over_own_life = values_for_region['moderate_or_much_control_over_own_life']
            difficulty_getting_around = values_for_region['difficulty_getting_around']
            serious_noise_nuisance_from_neighbours = values_for_region['serious_noise_nuisance_from_neighbours']

            stress_avg = ((moderate_or_much_control_over_own_life + (1 - difficulty_getting_around) + (1 - serious_noise_nuisance_from_neighbours)) / 3) * 1000
            vioscore_by_category[category] = float(stress_avg.iloc[0])

    vioscore_by_category['health_vioscore'] = (sum(value for value in vioscore_by_category.values()) / len(vioscore_by_category.keys())) * 0.7

    return vioscore_by_category

region_code_to_select = 'NL00'
values_for_region = get_values_for_region(region_code_to_select, categories)
print(values_for_region)